In [47]:
import os
import pickle
import copy
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

import matplotlib.pyplot as plt

In [48]:
class label_encoder(object):
    def fit_pd(self,df,cols=[]):
        if len(cols) == 0:
            cols = df.columns
        self.class_index = {}
        for f in cols:
            uf = df[f].unique()
            self.class_index[f] = {}
            index = 1
            for item in uf:
                self.class_index[f][item] = index
                index += 1
    
    def fit_transform_pd(self,df,cols=[]):
        if len(cols) == 0:
            cols = df.columns
        newdf = copy.deepcopy(df)
        self.class_index = {}
        for f in cols:
            uf = df[f].unique()
            self.class_index[f] = {}
            index = 1
            for item in uf:
                self.class_index[f][item] = index
                index += 1
                
            newdf[f] = df[f].apply(lambda d: self.update_label(f,d))
        return newdf
    
    def transform_pd(self,df,cols=[]):
        newdf = copy.deepcopy(df)
        if len(cols) == 0:
            cols = df.columns
        for f in cols:
            if f in self.class_index:
                newdf[f] = df[f].apply(lambda d: self.update_label(f,d))
        return newdf
                
    def update_label(self,f,x):
        try:
            return self.class_index[f][x]
        except:
            self.class_index[f][x] = max(self.class_index[f].values())+1
            return self.class_index[f][x]

In [49]:
def min_max_scaler(df, numerical_columns, scaler_path):
    df = df.loc[:, numerical_columns].copy()
    scalers = {}
    mm_sclaers = []
    
    if not os.path.exists(scaler_path):
        print('Make a New Min Max Scaler')

       
        for col in numerical_columns:
            mm_sclaer = MinMaxScaler()

            
            mm_sclaer.fit(df.loc[:, [col]])
            mm_sclaers.append(mm_sclaer)

            
            df.loc[:, col] = mm_sclaer.transform(df.loc[:, [col]])
            scalers[col] = mm_sclaer
        f = open(scaler_path, 'wb')
        pickle.dump(scalers, f)

    else:
        print('Normalize with existing Min Max Scaler')
        f = open(scaler_path, 'rb')
        scalers = pickle.load(f)

        for col in numerical_columns:
            mm_scaler = scalers[col]
            df.loc[:, col] = mm_scaler.transform(df.loc[:, [col]])

    return df, scalers

In [50]:
df_train = pd.read_csv(r"C:\Users\las\Desktop\pfe\fraud\data\train.csv")


In [51]:
df_valid = pd.read_csv(r"C:\Users\las\Desktop\pfe\fraud\data\valid.csv")
df_test = pd.read_csv(r"C:\Users\las\Desktop\pfe\fraud\data\test.csv")

In [52]:
df_train

,Declaration ID,Date,Office ID,Process Type,Import Type,Import Use,Payment Type,Mode of Transport,Declarant ID,Importer ID,...,HS6 Code,Country of Departure,Country of Origin,Tax Rate,Tax Type,Country of Origin Indicator,Net Mass,Item Price,Fraud,Critical Fraud
0,97061800,2020-01-01,30,B,11,21,11,10,ZZR1LT6,QLRUBN9,...,440890,BE,BE,0.0,FEU1,G,108.0,3.722544e+05,0,0
1,85945189,2020-01-01,40,B,11,21,11,40,SWF9S4X,7JD1S2X,...,690722,CN,CN,8.0,A,Y,11352.0,3.757512e+05,0,0
2,77854751,2020-01-01,20,B,11,21,11,40,X4XT6P8,WI9O3I5,...,620822,CN,CN,5.2,FCN1,B,20.7,5.353020e+03,0,0
3,46925060,2020-01-01,40,B,11,21,43,40,K7LCQTZ,6LI9721,...,940350,VN,VN,0.0,C,Y,9218.0,1.477645e+06,0,0
4,34131149,2020-01-01,20,B,11,21,11,10,1HMVIVH,RZ871V1,...,71080,VN,VN,27.0,A,B,24000.0,6.364800e+06,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100842,25225594,2021-06-30,40,B,87,21,18,40,BUK2SI8,Y7K2P0D,...,844399,SG,CN,0.0,C,Y,2.2,1.463000e+02,0,0
100843,17739793,2021-06-30,40,B,11,21,11,10,44F36R4,MIDF9C7,...,960329,CN,CN,3.2,FCN1,B,121.2,3.842040e+03,0,0
100844,70427265,2021-06-30,33,B,11,21,11,50,EJW9AY4,ZSST0QO,...,30749,CN,CN,12.0,FCN1,Y,21450.0,1.186078e+08,0,0
100845,60306302,2021-06-30,20,B,11,21,11,10,7GHJ33J,S6BWQ6D,...,851762,CN,CN,0.0,C,B,25.4,6.910349e+05,0,0


In [53]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100847 entries, 0 to 100846
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Declaration ID               100847 non-null  int64  
 1   Date                         100847 non-null  object 
 2   Office ID                    100847 non-null  int64  
 3   Process Type                 100847 non-null  object 
 4   Import Type                  100847 non-null  int64  
 5   Import Use                   100847 non-null  int64  
 6   Payment Type                 100847 non-null  int64  
 7   Mode of Transport            100847 non-null  int64  
 8   Declarant ID                 100847 non-null  object 
 9   Importer ID                  100847 non-null  object 
 10  Seller ID                    91774 non-null   object 
 11  Courier ID                   20239 non-null   object 
 12  HS6 Code                     100847 non-null  int64  
 13 

In [54]:


df_train['Date'] = pd.to_datetime(df_train['Date'], errors='coerce') 

df_train = df_train.astype({
    'Declaration ID': 'str',
    'Office ID': 'str',
    'Process Type': 'str',
    'Import Type': 'str',
    'Import Use': 'str',
    'Payment Type': 'str',
    'Mode of Transport': 'str',
    'HS6 Code': 'str',
    'Fraud': 'str',
    'Critical Fraud': 'str',
    'Item Price': 'float64'
})

df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100847 entries, 0 to 100846
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   Declaration ID               100847 non-null  object        
 1   Date                         100847 non-null  datetime64[ns]
 2   Office ID                    100847 non-null  object        
 3   Process Type                 100847 non-null  object        
 4   Import Type                  100847 non-null  object        
 5   Import Use                   100847 non-null  object        
 6   Payment Type                 100847 non-null  object        
 7   Mode of Transport            100847 non-null  object        
 8   Declarant ID                 100847 non-null  object        
 9   Importer ID                  100847 non-null  object        
 10  Seller ID                    91774 non-null   object        
 11  Courier ID                

In [55]:
df_valid['Date'] = pd.to_datetime(df_valid['Date'], errors='coerce')
df_valid = df_valid.astype({'Declaration ID':'str', 'Office ID':'str', 'Process Type':'str', 
              'Import Type':'str', 'Import Use':'str', 'Payment Type':'str', 'Mode of Transport':'str', 
              'HS6 Code':'str', 'Fraud':'str', 'Critical Fraud':'str', 'Item Price':'float64'})
df_test['Date'] = pd.to_datetime(df_test['Date'], errors='coerce')
df_test=df_test.astype({'Declaration ID':'str', 'Office ID':'str', 'Process Type':'str', 
              'Import Type':'str', 'Import Use':'str', 'Payment Type':'str', 'Mode of Transport':'str', 
              'HS6 Code':'str', 'Fraud':'str', 'Critical Fraud':'str', 'Item Price':'float64'})

In [56]:
df_train = df_train.fillna(0)
df_valid = df_valid.fillna(0)
df_test = df_test.fillna(0)

In [57]:
os.makedirs('./encoder/', exist_ok = True)

In [58]:
numerical_columns = ['Net Mass', 'Item Price']
train_numerical_xs, scaler = min_max_scaler(df_train, numerical_columns, scaler_path='./encoder/min_max_scaler_eng.pkl')
valid_numerical_xs, _ = min_max_scaler(df_valid, numerical_columns, scaler_path='./encoder/min_max_scaler_eng.pkl')

Normalize with existing Min Max Scaler
Normalize with existing Min Max Scaler


In [59]:

X_train_discrete = df_train[['Office ID', 'Process Type', 'Import Type', 'Import Use',
               'Payment Type', 'Mode of Transport', 'Declarant ID', 'Importer ID',
               'Seller ID', 'Courier ID', 'HS6 Code', 'Country of Departure',
               'Country of Origin', 'Tax Rate', 'Tax Type',
               'Country of Origin Indicator']]

X_valid_discrete = df_valid[['Office ID', 'Process Type', 'Import Type', 'Import Use',
               'Payment Type', 'Mode of Transport', 'Declarant ID', 'Importer ID',
               'Seller ID', 'Courier ID', 'HS6 Code', 'Country of Departure',
               'Country of Origin', 'Tax Rate', 'Tax Type',
               'Country of Origin Indicator']]



y_train = df_train['Fraud']
y_valid = df_valid['Fraud']

In [60]:

encoder = label_encoder()
encoder.fit_pd(X_train_discrete)
encoder.transform_pd(X_train_discrete)

,Office ID,Process Type,Import Type,Import Use,Payment Type,Mode of Transport,Declarant ID,Importer ID,Seller ID,Courier ID,HS6 Code,Country of Departure,Country of Origin,Tax Rate,Tax Type,Country of Origin Indicator
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,2,1,1,1,1,2,2,2,2,1,2,2,2,2,2,2
2,3,1,1,1,1,2,3,3,3,1,3,2,2,3,3,3
3,2,1,1,1,2,2,4,4,4,1,4,3,3,1,4,2
4,3,1,1,1,1,1,5,5,5,1,5,3,3,4,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100842,2,1,11,1,3,2,468,436,2563,7,208,4,2,1,4,2
100843,2,1,1,1,1,1,67,6989,3629,1,1099,2,2,9,3,3
100844,21,1,1,1,1,3,511,7459,3805,52,2489,2,2,6,3,2
100845,3,1,1,1,1,1,116,954,7974,1,12,2,2,1,4,3


In [61]:

filename = './encoder/label_encoder.pkl'

output = open(filename,'wb')
pickle.dump(encoder, output)
output.close()

In [62]:

encoding_train = encoder.transform_pd(X_train_discrete)
encoding_valid = encoder.transform_pd(X_valid_discrete)

In [63]:


X_train_concat = pd.concat([encoding_train, train_numerical_xs], axis=1)
X_valid_concat = pd.concat([encoding_valid, valid_numerical_xs], axis=1)

In [64]:
X_train_concat

,Office ID,Process Type,Import Type,Import Use,Payment Type,Mode of Transport,Declarant ID,Importer ID,Seller ID,Courier ID,HS6 Code,Country of Departure,Country of Origin,Tax Rate,Tax Type,Country of Origin Indicator,Net Mass,Item Price
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.799250e-06,5.283441e-05
1,2,1,1,1,1,2,2,2,2,1,2,2,2,2,2,2,1.891212e-04,5.333071e-05
2,3,1,1,1,1,2,3,3,3,1,3,2,2,3,3,3,3.448563e-07,7.597590e-07
3,2,1,1,1,2,2,4,4,4,1,4,3,3,1,4,2,1.535693e-04,2.097236e-04
4,3,1,1,1,1,1,5,5,5,1,5,3,3,4,2,3,3.998334e-04,9.033619e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100842,2,1,11,1,3,2,468,436,2563,7,208,4,2,1,4,2,3.665140e-08,2.076449e-08
100843,2,1,1,1,1,1,67,6989,3629,1,1099,2,2,9,3,3,2.019159e-06,5.453042e-07
100844,21,1,1,1,1,3,511,7459,3805,52,2489,2,2,6,3,2,3.573511e-04,1.683411e-02
100845,3,1,1,1,1,1,116,954,7974,1,12,2,2,1,4,3,4.231570e-07,9.807922e-05


In [65]:
X_valid_concat

,Office ID,Process Type,Import Type,Import Use,Payment Type,Mode of Transport,Declarant ID,Importer ID,Seller ID,Courier ID,HS6 Code,Country of Departure,Country of Origin,Tax Rate,Tax Type,Country of Origin Indicator,Net Mass,Item Price
0,2,1,1,5,1,2,609,8431,378,1,312,2,2,44,3,5,1.501374e-05,5.891449e-05
1,3,1,1,1,1,1,460,7715,10840,1,553,10,2,1,3,5,6.663890e-06,4.331731e-06
2,1,1,1,1,4,1,62,593,2143,1,88,2,2,9,3,4,5.647647e-06,3.704823e-07
3,3,1,1,1,1,2,812,4013,10841,1,197,2,2,12,2,5,4.997918e-07,3.517048e-07
4,8,1,1,1,1,2,1227,6612,4445,1,482,6,5,48,8,3,4.997918e-08,1.222383e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20054,2,1,1,1,2,2,176,853,5,1,976,9,7,1,4,5,4.388172e-06,1.650455e-04
20055,1,1,1,1,2,2,243,2294,4566,1,5,2,2,4,2,3,1.999167e-03,4.516809e-03
20056,1,1,1,1,1,1,269,12890,4999,1,1282,8,8,2,2,2,3.831737e-07,2.908917e-06
20057,2,1,1,1,1,2,26,1949,12161,1,930,2,2,18,3,2,1.857559e-05,6.353857e-05


In [66]:
os.makedirs('./data_enc/', exist_ok = True)
X_train_concat.to_csv('./data_enc/encoded_train.csv', index=False)
X_valid_concat.to_csv('./data_enc/encoded_valid.csv', index=False)

In [67]:
y_train.to_csv('./data_enc/y_train.csv', index=False)
y_valid.to_csv('./data_enc/y_valid.csv', index=False)

In [68]:



y_test = df_test['Fraud']


numerical_columns = ['Net Mass', 'Item Price']
test_numerical_xs, _ = min_max_scaler(df_test, numerical_columns, scaler_path='./encoder/min_max_scaler_eng.pkl')


X_test_discrete = df_test[['Office ID', 'Process Type', 'Import Type', 'Import Use',
               'Payment Type', 'Mode of Transport', 'Declarant ID', 'Importer ID',
               'Seller ID', 'Courier ID', 'HS6 Code', 'Country of Departure',
               'Country of Origin', 'Tax Rate', 'Tax Type',
               'Country of Origin Indicator']]

filename = './encoder/label_encoder.pkl'
pkl_file = open(filename, 'rb')
encoder = pickle.load(pkl_file) 
pkl_file.close()
encoding_test = encoder.transform_pd(X_test_discrete)

X_test_concat = pd.concat([encoding_test, test_numerical_xs], axis=1)

Normalize with existing Min Max Scaler


In [69]:
X_test_concat.to_csv('./data_enc/encoded_test.csv', index=False)
y_test.to_csv('./data_enc/y_test.csv', index=False)